In [1]:
import pandas as pd
import numpy as np
import requests
import urllib.request as request
from sklearn.preprocessing import OneHotEncoder
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from keras.preprocessing.text import Tokenizer
from nltk.stem.lancaster import LancasterStemmer
import json 
import random
import re
import string
from nltk.stem.snowball import SnowballStemmer
from keras.models import Sequential,load_model
from keras.layers import Dense, LSTM, Bidirectional, Embedding, Dropout
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.sequence import pad_sequences
import nltk
nltk.download('stopwords')
stopword = set(stopwords.words('english')) 

Using TensorFlow backend.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kb\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Loading data
### Loading it directly from GitHub. No need to save in local!

In [2]:
def data_loader(url):
    resp = requests.get(url)
    data = json.loads(resp.text)
    print("Data loaded successfully!")
    print("The dataset has following keys =",data.keys())
    return data

# Data Pre-processing

In [3]:

def data_preprocessing(data):
    # extracting test ,train and validation data from the .json file into dataframe
    full_test_data = pd.DataFrame(data['test'],columns=['text','label'])
    full_train_data = pd.DataFrame(data['train'],columns=['text','label'])
    full_val_data = pd.DataFrame(data['val'],columns=['text','label'])

    #identifying all labels in the dataset and storing it in a list.
    label_list=full_train_data['label'].unique().tolist()

    # Using random getting 20 random labels from the total labels

    k=0
    d={}
    while(k!=20):
        x = random.randint(0,len(label_list))
        if x in d:
            continue
        d[x]=x
        k+=1
    for i in d:
        selected.append(label_list[i-1])

    #Getting the train data with the selected labels
    train_20labels=pd.DataFrame()
    for i in selected:
        df=full_train_data[full_train_data['label']==i]
        frames=[train_20labels,df]
        train_20labels=pd.concat(frames)

    #Getting the test data with the selected labels
    test_20labels=pd.DataFrame()
    for i in selected:
        df=full_test_data[full_test_data['label']==i]
        frames=[test_20labels,df]
        test_20labels=pd.concat(frames)
    
    #Getting the val data with the selected labels
    val_20labels=pd.DataFrame()
    for i in selected:
        df=full_val_data[full_val_data['label']==i]
        frames=[val_20labels,df]
        val_20labels=pd.concat(frames)
    return train_20labels,test_20labels,val_20labels





## Data Cleanising

In [4]:
def data_cleansing():
    # Removing punctuations
    def punctuation_removal(text):
        no_punct=[words for words in text if words not in string.punctuation]
        words_wo_punct=''.join(no_punct)
        return words_wo_punct

    # Tokenize strings
    def tokenize(text):
        split=re.split("\W+",text) 
        return split

    # Remove stopwords
    def remove_stopwords(text):
        text=[word for word in text if word not in stopword]
        return text

    #Defining Stemmer
    def stemmer(text):
        s_stemmer = SnowballStemmer(language='english')
        for word in text:
            text=[s_stemmer.stem(word) for word in text]
        return text
    train_20labels['text']=train_20labels['text'].apply(lambda x: punctuation_removal(x))
    train_20labels['text']=train_20labels['text'].apply(lambda x: tokenize(x.lower()))
    train_20labels['text']=train_20labels['text'].apply(lambda x: remove_stopwords(x))
    #train_20labels['text']=train_20labels['text'].apply(lambda x: stemmer(x))

    test_20labels['text']=test_20labels['text'].apply(lambda x: punctuation_removal(x))
    test_20labels['text']=test_20labels['text'].apply(lambda x: tokenize(x.lower()))
    test_20labels['text']=test_20labels['text'].apply(lambda x: remove_stopwords(x))
    #test_20labels['text']=test_20labels['text'].apply(lambda x: stemmer(x))

    val_20labels['text']=val_20labels['text'].apply(lambda x: punctuation_removal(x))
    val_20labels['text']=val_20labels['text'].apply(lambda x: tokenize(x.lower()))
    val_20labels['text']=val_20labels['text'].apply(lambda x: remove_stopwords(x))
    #val_20labels['text']=val_20labels['text'].apply(lambda x: stemmer(x))
    train_20labels['label']=train_20labels['label'].str.replace('_','')
    test_20labels['label']=test_20labels['label'].str.replace('_','')
    val_20labels['label']=val_20labels['label'].str.replace('_','')

# Data Representation and feature selection

In [5]:
d={}
def data_representation():
    def changing1(t,x):
        a=[0 for i in range(20)]
        a[t.word_index.get(x)-1]=1
        d[t.word_index.get(x)-1]=x
        return a

    def changing(x):
        q=t.texts_to_sequences(x)
        r=[]
        if len(q)!=0:
            for i in q:
                r.append(i[0])
        c=max_length-len(r)
        z=[0 for i in range(c)]
        r=r+z
        return np.asarray(r)

    r=pd.DataFrame()
    frames=[train_20labels['text'],test_20labels['text'],val_20labels['text']]
    r['text']=pd.concat(frames)
    s=set()

    ##Input tokenizer for vectorizing features ie. text data for all test,train and val dataframe
    t = Tokenizer()
    t.fit_on_texts(r['text'])
    for i in range(len(r['text'])):
        for j in range(len(r['text'].iloc[i])):
            s.add(r['text'].iloc[i][j])

    vocab_size=len(s)
    max_length=int(train_20labels['text'].str.len().max())

    #applying the changes to train/test/val datarame
    train_20labels['matrix']= train_20labels['text'].apply(lambda x:changing(x))
    test_20labels['matrix']= test_20labels['text'].apply(lambda x: changing(x))
    val_20labels['matrix']= val_20labels['text'].apply(lambda x: changing(x))


    ##Output tokenizer for encoding labels

    ot=Tokenizer()
    ot.fit_on_texts(train_20labels['label'].values.tolist())


    #applying the changes to train/test/val datarame
    train_20labels['elabel']= train_20labels['label'].apply(lambda x: changing1(ot,x))
    test_20labels['elabel']= test_20labels['label'].apply(lambda x: changing1(ot,x))
    val_20labels['elabel']= val_20labels['label'].apply(lambda x: changing1(ot,x))
    train_20labels['elabel'] = np.array(train_20labels['elabel']).reshape(len(train_20labels['elabel']), 1)
    test_20labels['elabel'] = np.array(test_20labels['elabel']).reshape(len(test_20labels['elabel']), 1)
    val_20labels['elabel'] = np.array(val_20labels['elabel']).reshape(len(val_20labels['elabel']), 1)

    return vocab_size,max_length

In [6]:
def data_preperation():
    train_X=train_20labels['matrix']
    train_Y =train_20labels['elabel']
    val_X =val_20labels['matrix']
    val_Y =val_20labels['elabel']
    test_X=train_20labels['matrix']
    test_Y =train_20labels['elabel']

    # reshaping train_X and train_Y
    train_X=train_X.to_numpy()
    train_X=np.vstack(train_X)
    train_Y=train_Y.to_numpy()
    train_Y=np.vstack(train_Y)

    # reshaping val_X and val_Y
    val_X=val_X.to_numpy()
    val_X=np.vstack(val_X)
    val_Y=val_Y.to_numpy()
    val_Y=np.vstack(val_Y)

    # reshaping test_X and test_Y
    test_X=test_X.to_numpy()
    test_X=np.vstack(test_X)
    test_Y=test_Y.to_numpy()
    test_Y=np.vstack(test_Y)
    return train_X,test_X,val_X,train_Y,test_Y,val_Y

In [7]:
def LSTM_model(vocab_size, max_length,train_X,val_X,train_Y,val_Y):
    def create_model(vocab_size, max_length):
        model = Sequential()
        model.add(Embedding(vocab_size, 64, input_length = max_length, trainable = False))
        model.add(Bidirectional(LSTM(256)))
        model.add(Dense(32, activation = "relu"))
        model.add(Dropout(0.5))
        model.add(Dense(20, activation = "softmax"))
        return model

    model = create_model(vocab_size+1, max_length)
    model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
    model.summary()
    filename = 'model.h5'
    checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

    hist = model.fit(train_X, train_Y, epochs = 200, batch_size = 16, shuffle=True, validation_data = (val_X, val_Y), callbacks = [checkpoint])

In [8]:
def predict(test_X):
    def predictions():
        model = load_model("model.h5")
        pred=model.predict(test_X)
        return pred
    pred=predictions()
    def get_final_output(pred, classes):
      for i in range(len(pred)):
        predictions = pred[i]
        classes = np.array(classes)
        ids = np.argsort(-predictions)
        classes = classes[ids]
        predictions = -np.sort(-predictions)
        print("Testing for statement :", question[i])
        true=''
        for j in range(len(test_Y[i])):
            if test_Y[i][j]==1:
                true=d[j+1]
                break
        print('\n Results are:')
        for i in range(pred.shape[1]):
            print("%s has confidence = %s" % (classes[i], (predictions[i])))
        print('------------------------------------------------------------------------')
    p=list(selected)
    for i in range(len(selected)):
        p[i]=p[i].replace('_','')
    get_final_output(pred, p)


In [9]:
data=data_loader("https://raw.githubusercontent.com/clinc/oos-eval/master/data/data_full.json")
selected=[]
train_20labels,test_20labels,val_20labels = data_preprocessing(data)
finaldf=test_20labels
question=[]
for i in range(len(finaldf)):
    question.append(finaldf['text'].iloc[i])
print("\nRandomly Selected 20 labels from in-scope content are =\n", selected)
data_cleansing()
vocab_size,max_length=data_representation()
train_X,test_X,val_X,train_Y,test_Y,val_Y=data_preperation()
LSTM_model(vocab_size, max_length,train_X,val_X,train_Y,val_Y)


Data loaded successfully!
The dataset has following keys = dict_keys(['oos_val', 'val', 'train', 'oos_test', 'test', 'oos_train'])

Randomly Selected 20 labels from in-scope content are =
 ['nutrition_info', 'who_made_you', 'credit_limit_change', 'make_call', 'flip_coin', 'time', 'pay_bill', 'spending_history', 'book_flight', 'rewards_balance', 'restaurant_suggestion', 'cancel', 'calendar_update', 'change_accent', 'insurance', 'fun_fact', 'what_can_i_ask_you', 'meeting_schedule', 'smart_home', 'plug_type']
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 15, 64)            118848    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 512)               657408    
_________________________________________________________________
dense_1 (Dense)              (None, 32)                16416     
____

2000/2000 [==============================] - 10s 5ms/step - loss: 0.7215 - accuracy: 0.7670 - val_loss: 0.9349 - val_accuracy: 0.7600

Epoch 00031: val_loss did not improve from 0.90510
Epoch 32/200
2000/2000 [==============================] - 11s 5ms/step - loss: 0.7186 - accuracy: 0.7630 - val_loss: 0.9822 - val_accuracy: 0.7375

Epoch 00032: val_loss did not improve from 0.90510
Epoch 33/200
2000/2000 [==============================] - 10s 5ms/step - loss: 0.6737 - accuracy: 0.7695 - val_loss: 0.9167 - val_accuracy: 0.7725

Epoch 00033: val_loss did not improve from 0.90510
Epoch 34/200
2000/2000 [==============================] - 8s 4ms/step - loss: 0.6700 - accuracy: 0.7855 - val_loss: 0.9907 - val_accuracy: 0.7350

Epoch 00034: val_loss did not improve from 0.90510
Epoch 35/200
2000/2000 [==============================] - 11s 5ms/step - loss: 0.7404 - accuracy: 0.7650 - val_loss: 0.9418 - val_accuracy: 0.7625

Epoch 00035: val_loss did not improve from 0.90510
Epoch 36/200
2000/2

2000/2000 [==============================] - 11s 6ms/step - loss: 0.2870 - accuracy: 0.8930 - val_loss: 1.1578 - val_accuracy: 0.7850

Epoch 00072: val_loss did not improve from 0.90510
Epoch 73/200
2000/2000 [==============================] - 12s 6ms/step - loss: 0.2807 - accuracy: 0.8965 - val_loss: 1.2064 - val_accuracy: 0.7800

Epoch 00073: val_loss did not improve from 0.90510
Epoch 74/200
2000/2000 [==============================] - 11s 6ms/step - loss: 0.2979 - accuracy: 0.8980 - val_loss: 1.2498 - val_accuracy: 0.7400

Epoch 00074: val_loss did not improve from 0.90510
Epoch 75/200
2000/2000 [==============================] - 10s 5ms/step - loss: 0.4152 - accuracy: 0.8580 - val_loss: 1.2540 - val_accuracy: 0.7700

Epoch 00075: val_loss did not improve from 0.90510
Epoch 76/200
2000/2000 [==============================] - 12s 6ms/step - loss: 0.3933 - accuracy: 0.8730 - val_loss: 1.2398 - val_accuracy: 0.7400

Epoch 00076: val_loss did not improve from 0.90510
Epoch 77/200
2000/

2000/2000 [==============================] - 8s 4ms/step - loss: 0.1520 - accuracy: 0.9500 - val_loss: 1.2746 - val_accuracy: 0.8100

Epoch 00113: val_loss did not improve from 0.90510
Epoch 114/200
2000/2000 [==============================] - 8s 4ms/step - loss: 0.1548 - accuracy: 0.9480 - val_loss: 1.9584 - val_accuracy: 0.8025

Epoch 00114: val_loss did not improve from 0.90510
Epoch 115/200
2000/2000 [==============================] - 8s 4ms/step - loss: 0.1432 - accuracy: 0.9495 - val_loss: 1.7946 - val_accuracy: 0.8050

Epoch 00115: val_loss did not improve from 0.90510
Epoch 116/200
2000/2000 [==============================] - 8s 4ms/step - loss: 0.1460 - accuracy: 0.9430 - val_loss: 1.5714 - val_accuracy: 0.7975

Epoch 00116: val_loss did not improve from 0.90510
Epoch 117/200
2000/2000 [==============================] - 8s 4ms/step - loss: 0.2398 - accuracy: 0.9255 - val_loss: 1.7600 - val_accuracy: 0.7450

Epoch 00117: val_loss did not improve from 0.90510
Epoch 118/200
2000/

2000/2000 [==============================] - 9s 4ms/step - loss: 0.1193 - accuracy: 0.9600 - val_loss: 1.3877 - val_accuracy: 0.8125

Epoch 00154: val_loss did not improve from 0.90510
Epoch 155/200
2000/2000 [==============================] - 9s 4ms/step - loss: 0.1171 - accuracy: 0.9635 - val_loss: 1.5329 - val_accuracy: 0.8175

Epoch 00155: val_loss did not improve from 0.90510
Epoch 156/200
2000/2000 [==============================] - 9s 4ms/step - loss: 0.1110 - accuracy: 0.9620 - val_loss: 1.4972 - val_accuracy: 0.8100

Epoch 00156: val_loss did not improve from 0.90510
Epoch 157/200
2000/2000 [==============================] - 9s 4ms/step - loss: 0.0873 - accuracy: 0.9700 - val_loss: 1.5981 - val_accuracy: 0.8150

Epoch 00157: val_loss did not improve from 0.90510
Epoch 158/200
2000/2000 [==============================] - 8s 4ms/step - loss: 0.1145 - accuracy: 0.9590 - val_loss: 1.6228 - val_accuracy: 0.8175

Epoch 00158: val_loss did not improve from 0.90510
Epoch 159/200
2000/

2000/2000 [==============================] - 9s 4ms/step - loss: 0.1830 - accuracy: 0.9640 - val_loss: 1.5760 - val_accuracy: 0.7875

Epoch 00195: val_loss did not improve from 0.90510
Epoch 196/200
2000/2000 [==============================] - 8s 4ms/step - loss: 0.1441 - accuracy: 0.9495 - val_loss: 1.3854 - val_accuracy: 0.8125

Epoch 00196: val_loss did not improve from 0.90510
Epoch 197/200
2000/2000 [==============================] - 8s 4ms/step - loss: 0.0906 - accuracy: 0.9675 - val_loss: 1.4942 - val_accuracy: 0.8200

Epoch 00197: val_loss did not improve from 0.90510
Epoch 198/200
2000/2000 [==============================] - 8s 4ms/step - loss: 0.1204 - accuracy: 0.9645 - val_loss: 1.3905 - val_accuracy: 0.8075

Epoch 00198: val_loss did not improve from 0.90510
Epoch 199/200
2000/2000 [==============================] - 8s 4ms/step - loss: 0.1015 - accuracy: 0.9595 - val_loss: 1.5066 - val_accuracy: 0.8375

Epoch 00199: val_loss did not improve from 0.90510
Epoch 200/200
2000/

In [10]:
predict(test_X)

Testing for statement : tell me nutritional info for brocoli

 Results are:
nutritioninfo has confidence = 0.78408664
calendarupdate has confidence = 0.12832645
funfact has confidence = 0.015427505
plugtype has confidence = 0.011979096
insurance has confidence = 0.009584684
meetingschedule has confidence = 0.009341037
bookflight has confidence = 0.008164719
changeaccent has confidence = 0.007939502
restaurantsuggestion has confidence = 0.00599783
smarthome has confidence = 0.0055554644
cancel has confidence = 0.0051058265
paybill has confidence = 0.0018767099
rewardsbalance has confidence = 0.0016486249
time has confidence = 0.0013714082
whatcaniaskyou has confidence = 0.0013707714
spendinghistory has confidence = 0.001216427
makecall has confidence = 0.00042316757
flipcoin has confidence = 0.00041593128
whomadeyou has confidence = 8.736413e-05
creditlimitchange has confidence = 8.093928e-05
------------------------------------------------------------------------
Testing for statement 

whatcaniaskyou has confidence = 0.04645409
meetingschedule has confidence = 0.041448426
restaurantsuggestion has confidence = 0.038134713
funfact has confidence = 0.026506392
plugtype has confidence = 0.025679559
whomadeyou has confidence = 0.017266387
calendarupdate has confidence = 0.009280582
spendinghistory has confidence = 0.008804489
makecall has confidence = 0.0041446383
nutritioninfo has confidence = 0.0040047374
insurance has confidence = 0.0028266043
creditlimitchange has confidence = 0.00146151
changeaccent has confidence = 0.00067640236
smarthome has confidence = 0.0006641285
cancel has confidence = 0.0005054915
time has confidence = 0.00029633733
------------------------------------------------------------------------
Testing for statement : can my credit limit be increased by 500 dollars

 Results are:
meetingschedule has confidence = 0.529031
cancel has confidence = 0.1306989
flipcoin has confidence = 0.10611212
creditlimitchange has confidence = 0.09030589
spendinghisto

Testing for statement : can you please do a simulated coin flip for me ai

 Results are:
insurance has confidence = 0.4228015
calendarupdate has confidence = 0.32223538
whatcaniaskyou has confidence = 0.11716608
makecall has confidence = 0.09231405
funfact has confidence = 0.020988397
paybill has confidence = 0.005718957
spendinghistory has confidence = 0.0043617436
meetingschedule has confidence = 0.0036557335
cancel has confidence = 0.0028786794
plugtype has confidence = 0.0027028706
time has confidence = 0.0021583654
restaurantsuggestion has confidence = 0.0008017981
changeaccent has confidence = 0.0006740941
nutritioninfo has confidence = 0.0005261289
creditlimitchange has confidence = 0.00030750438
bookflight has confidence = 0.0002601856
whomadeyou has confidence = 0.00021176858
rewardsbalance has confidence = 0.00014546176
smarthome has confidence = 8.938056e-05
flipcoin has confidence = 1.8970615e-06
------------------------------------------------------------------------
Testi


 Results are:
whatcaniaskyou has confidence = 0.5068953
paybill has confidence = 0.24887018
flipcoin has confidence = 0.11548841
smarthome has confidence = 0.073786944
cancel has confidence = 0.03898783
meetingschedule has confidence = 0.007402387
plugtype has confidence = 0.0028472631
insurance has confidence = 0.0017242756
rewardsbalance has confidence = 0.0015672251
restaurantsuggestion has confidence = 0.00061101065
changeaccent has confidence = 0.0005959642
funfact has confidence = 0.00039334482
spendinghistory has confidence = 0.0003730009
nutritioninfo has confidence = 0.00013415469
makecall has confidence = 0.00011174089
bookflight has confidence = 9.77027e-05
whomadeyou has confidence = 4.045856e-05
time has confidence = 3.863021e-05
calendarupdate has confidence = 3.40451e-05
creditlimitchange has confidence = 2.7052258e-07
------------------------------------------------------------------------
Testing for statement : pay my utility bill from my bank account

 Results are:


Testing for statement : i want to fly to atlanta for under 300 from pittsburgh

 Results are:
whatcaniaskyou has confidence = 0.99904877
calendarupdate has confidence = 0.000651993
rewardsbalance has confidence = 0.0002974768
insurance has confidence = 6.978076e-07
creditlimitchange has confidence = 6.0279905e-07
funfact has confidence = 1.4119881e-07
meetingschedule has confidence = 1.0189673e-07
restaurantsuggestion has confidence = 1.002636e-07
spendinghistory has confidence = 1.7486723e-08
makecall has confidence = 6.509668e-09
paybill has confidence = 5.099858e-09
changeaccent has confidence = 2.2789102e-09
whomadeyou has confidence = 8.4696944e-10
flipcoin has confidence = 2.3218096e-10
smarthome has confidence = 2.132816e-10
cancel has confidence = 1.3920227e-10
time has confidence = 4.880526e-11
bookflight has confidence = 4.994061e-12
plugtype has confidence = 4.6183374e-12
nutritioninfo has confidence = 1.137696e-12
------------------------------------------------------------

cancel has confidence = 0.094115295
spendinghistory has confidence = 0.08844196
paybill has confidence = 0.06469618
makecall has confidence = 0.03917784
insurance has confidence = 0.03658693
plugtype has confidence = 0.03489794
flipcoin has confidence = 0.033603255
meetingschedule has confidence = 0.02681587
smarthome has confidence = 0.025332034
funfact has confidence = 0.023975769
whatcaniaskyou has confidence = 0.021869112
calendarupdate has confidence = 0.019891711
changeaccent has confidence = 0.017811697
rewardsbalance has confidence = 0.0071754074
time has confidence = 0.0037296575
creditlimitchange has confidence = 0.0032052847
nutritioninfo has confidence = 0.00131286
------------------------------------------------------------------------
Testing for statement : you can cancel the task

 Results are:
cancel has confidence = 0.8107586
calendarupdate has confidence = 0.07451973
creditlimitchange has confidence = 0.02535662
flipcoin has confidence = 0.022031069
bookflight has co

bookflight has confidence = 2.3970692e-10
whomadeyou has confidence = 1.1829067e-10
whatcaniaskyou has confidence = 8.5838045e-11
funfact has confidence = 7.507119e-11
cancel has confidence = 6.3908046e-11
time has confidence = 6.266477e-11
plugtype has confidence = 4.717823e-11
calendarupdate has confidence = 4.1506354e-11
creditlimitchange has confidence = 4.0729174e-11
nutritioninfo has confidence = 2.0463507e-12
flipcoin has confidence = 1.0051235e-13
meetingschedule has confidence = 7.1604484e-18
------------------------------------------------------------------------
Testing for statement : speak with a male british accent

 Results are:
smarthome has confidence = 0.99126
restaurantsuggestion has confidence = 0.0034587188
whatcaniaskyou has confidence = 0.0022810928
flipcoin has confidence = 0.0010529626
funfact has confidence = 0.00051590026
rewardsbalance has confidence = 0.00030058407
creditlimitchange has confidence = 0.00023188627
whomadeyou has confidence = 0.00019352202
nu

changeaccent has confidence = 4.4003816e-08
spendinghistory has confidence = 2.3933164e-09
creditlimitchange has confidence = 1.1037824e-09
plugtype has confidence = 9.633258e-10
cancel has confidence = 8.186464e-11
meetingschedule has confidence = 3.1639805e-11
nutritioninfo has confidence = 2.0507237e-12
time has confidence = 5.777386e-13
rewardsbalance has confidence = 4.1527236e-13
makecall has confidence = 1.4443615e-13
whatcaniaskyou has confidence = 6.837206e-14
bookflight has confidence = 3.8788695e-14
insurance has confidence = 1.6332923e-17
------------------------------------------------------------------------
Testing for statement : tell me what types of questions i can ask you

 Results are:
calendarupdate has confidence = 0.9998646
bookflight has confidence = 7.137495e-05
cancel has confidence = 5.311086e-05
smarthome has confidence = 7.750127e-06
makecall has confidence = 2.4532485e-06
paybill has confidence = 3.478351e-07
rewardsbalance has confidence = 2.749677e-07
wh

restaurantsuggestion has confidence = 0.012805116
whatcaniaskyou has confidence = 0.012615383
changeaccent has confidence = 0.011809481
calendarupdate has confidence = 0.0038528482
paybill has confidence = 0.00095024693
------------------------------------------------------------------------
Testing for statement : how do i plug stuff in in france

 Results are:
rewardsbalance has confidence = 0.9825764
spendinghistory has confidence = 0.013219954
restaurantsuggestion has confidence = 0.0018242571
calendarupdate has confidence = 0.0017606613
time has confidence = 0.00046479327
whatcaniaskyou has confidence = 7.044572e-05
changeaccent has confidence = 3.540528e-05
creditlimitchange has confidence = 3.2759275e-05
insurance has confidence = 8.780153e-06
cancel has confidence = 2.997796e-06
smarthome has confidence = 1.6132553e-06
funfact has confidence = 1.0192368e-06
paybill has confidence = 6.3997845e-07
meetingschedule has confidence = 2.1007442e-07
nutritioninfo has confidence = 1.485

IndexError: list index out of range